In [27]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode,MapNode
import nipype.interfaces.spm.utils as spmu
from nipype.interfaces import fsl
from nipype.interfaces.freesurfer import MRIConvert
from shutil import copyfile
from pandas import Series
from glob import glob
from os.path import abspath
from nipype import config, logging

In [28]:
proj_home = '/Volumes/group/proc/TIGERanalysis/qT1'
qt1dir = proj_home+'/subjDir'
output_dir = proj_home +'/processed'
workflow_dir = proj_home+'/workflows'
datasink_dir = proj_home +'/datasink'
subs_list = open(qt1dir + '/subjects.txt').read().splitlines()

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#Turn on debug node
config.enable_debug_mode()
logging.update_logging(config)

In [29]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = [('subject_id', subs_list)]


# Data grabber- select dti matfiles to convert
datasource1 = Node(DataGrabber(infields=['subject_id']),
                 base_directory = qt1dir,
                 template = '*-T1/wm.seg.mgz'
                 name = 'datasource')


In [30]:
convert = Node(MRIConvert(out_file = 'wm-seg.nii.gz',
                         out_type = 'niigz'),
              name = 'convert')

In [31]:
threshold = Node(fsl.Threshold(thresh = 4028,
                              direction = 'above'),
                name = 'threshold')

In [32]:
mask_flow = Workflow(name = 'mask_flow')
mask_flow.connect([(infosource, datasource1,[('subject_id','subject_id')]),
                   (datasource1, convert,[('subject_id','in_file')]),
                   (convert, threshold,[('out_file','in_file')]),
                   ])
mask_flow.base_dir = workflow_dir
mask_flow.write_graph(graph2use = 'flat')
mask_flow.run()

181128-19:57:59,420 nipype.workflow DEBUG:
	 (mask_flow.infosource, mask_flow.datasource): No edge data
181128-19:57:59,424 nipype.workflow DEBUG:
	 (mask_flow.infosource, mask_flow.datasource): new edge data: {'connect': [('subject_id', 'subject_id')]}
181128-19:57:59,426 nipype.workflow DEBUG:
	 (mask_flow.datasource, mask_flow.convert): No edge data
181128-19:57:59,428 nipype.workflow DEBUG:
	 (mask_flow.datasource, mask_flow.convert): new edge data: {'connect': [('subject_id', 'in_file')]}
181128-19:57:59,430 nipype.workflow DEBUG:
	 (mask_flow.convert, mask_flow.threshold): No edge data
181128-19:57:59,433 nipype.workflow DEBUG:
	 (mask_flow.convert, mask_flow.threshold): new edge data: {'connect': [('out_file', 'in_file')]}
181128-19:57:59,449 nipype.workflow DEBUG:
	 Creating flat graph for workflow: mask_flow
181128-19:57:59,456 nipype.workflow DEBUG:
	 expanding workflow: mask_flow
181128-19:57:59,459 nipype.workflow DEBUG:
	 processing node: mask_flow.infosource
181128-19:57:

ValueError: DataGrabber requires a value for input 'sort_filelist'. For a list of required inputs, see DataGrabber.help()

In [22]:
b0_flow = Workflow(name = 'b0_flow')
b0_flow.connect([(infosource,selectfiles,[('subject_id','subject_id')]),
                 (selectfiles, split, [('dti', 'in_file')]),
                 (split, datasink, [('out_files', 'split_dti')])
                ])

b0_flow.base_dir = workflow_dir
b0_flow.write_graph(graph2use = 'flat')
b0_flow.run()

180927-19:15:44,774 nipype.workflow DEBUG:
	 (b0_flow.infosource, b0_flow.selectfiles): No edge data
180927-19:15:44,779 nipype.workflow DEBUG:
	 (b0_flow.infosource, b0_flow.selectfiles): new edge data: {'connect': [('subject_id', 'subject_id')]}
180927-19:15:44,782 nipype.workflow DEBUG:
	 (b0_flow.selectfiles, b0_flow.split): No edge data
180927-19:15:44,786 nipype.workflow DEBUG:
	 (b0_flow.selectfiles, b0_flow.split): new edge data: {'connect': [('dti', 'in_file')]}
180927-19:15:44,789 nipype.workflow DEBUG:
	 (b0_flow.split, b0_flow.datasink): No edge data
180927-19:15:44,791 nipype.workflow DEBUG:
	 (b0_flow.split, b0_flow.datasink): new edge data: {'connect': [('out_files', 'split_dti')]}
180927-19:15:44,793 nipype.workflow DEBUG:
	 (b0_flow.strip, b0_flow.datasink): No edge data
180927-19:15:44,795 nipype.workflow DEBUG:
	 (b0_flow.strip, b0_flow.datasink): new edge data: {'connect': [('out_file', 'stripped_b0')]}
180927-19:15:44,884 nipype.workflow DEBUG:
	 Creating flat grap

180927-19:15:47,394 nipype.workflow DEBUG:
	 Performing depth first search
180927-19:15:47,628 nipype.workflow INFO:
	 Running serially.
180927-19:15:47,631 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_20-T1/selectfiles".
180927-19:15:47,634 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:15:47,697 nipype.workflow DEBUG:
	 [Node] Hashes: [('force_lists', False), ('raise_on_empty', True), ('sort_filelist', True), ('subject_id', '20-T1')], 393992d39f4cf34ba4c4b33917b7efa2, /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_20-T1/selectfiles/_0x393992d39f4cf34ba4c4b33917b7efa2.json, ['/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_20-T1/selectfiles/_0x393992d39f4cf34ba4c4b33917b7efa2.json']
180927-19:15:47,699 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "b0_flow.selectfiles".
180927-19:15:47,700 nipype.workflow DEBUG:
	 [Node] Rerunning cached

180927-19:15:52,709 nipype.utils DEBUG:
	 Creating directory /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split
180927-19:15:52,995 nipype.utils DEBUG:
	 Creating directory /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split/_report
180927-19:15:53,69 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split/_report/report.rst"
180927-19:15:53,602 nipype.workflow DEBUG:
	 copying files to wd [execute=True, linksonly=False]
180927-19:15:53,615 nipype.interface DEBUG:
	 dimension_t
180927-19:15:53,617 nipype.interface DEBUG:
	 in_file_/Volumes/iang/proc/TIGERanalysis/qT1/subjDir/19-T1/dti.nii.gz
180927-19:15:53,620 nipype.interface DEBUG:
	 out_base_name_dti_split
180927-19:15:53,744 nipype.workflow INFO:
	 [Node] Running "split" ("nipype.interfaces.fsl.utils.Split"), a CommandLine Interface with command:
fslsplit /Volumes/iang/proc/TIGERanalysis/qT1/sub

180927-19:17:59,603 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split/_report
180927-19:17:59,604 nipype.workflow DEBUG:
	 Removing files: 
180927-19:17:59,722 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split/result_split.pklz
180927-19:17:59,814 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_19-T1/split/_report/report.rst"
180927-19:17:59,857 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:17:59,958 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:17:59,961 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_18-T1/selectfiles".
180927-19:17:59,963 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:17:59,994 nipype.workflow DEBUG:
	 [Node] Directory not found "/V

180927-19:20:30,427 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_18-T1/split/_report
180927-19:20:30,428 nipype.workflow DEBUG:
	 Removing files: 
180927-19:20:30,554 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_18-T1/split/result_split.pklz
180927-19:20:30,783 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_18-T1/split/_report/report.rst"
180927-19:20:30,829 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:20:30,933 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:20:30,936 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_17-T1/selectfiles".
180927-19:20:30,939 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:20:30,971 nipype.workflow DEBUG:
	 [Node] Directory not found "/V

...

180927-19:35:57,558 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_11-T1/split/_report
180927-19:35:57,560 nipype.workflow DEBUG:
	 Removing files: 
180927-19:35:57,686 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_11-T1/split/result_split.pklz
180927-19:35:57,778 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_11-T1/split/_report/report.rst"
180927-19:35:57,824 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:35:57,932 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:35:57,935 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_10-T1/selectfiles".
180927-19:35:57,937 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:35:57,964 nipype.workflow DEBUG:
	 [Node] Directory not found "/V

180927-19:38:56,32 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_10-T1/split/_report
180927-19:38:56,34 nipype.workflow DEBUG:
	 Removing files: 
180927-19:38:56,158 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_10-T1/split/result_split.pklz
180927-19:38:56,257 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_10-T1/split/_report/report.rst"
180927-19:38:56,305 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:38:56,416 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:38:56,419 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_08-T1/selectfiles".
180927-19:38:56,420 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:38:56,448 nipype.workflow DEBUG:
	 [Node] Directory not found "/Vol

180927-19:41:45,825 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_08-T1/split/_report
180927-19:41:45,827 nipype.workflow DEBUG:
	 Removing files: 
180927-19:41:45,947 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_08-T1/split/result_split.pklz
180927-19:41:46,52 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_08-T1/split/_report/report.rst"
180927-19:41:46,99 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:41:46,221 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:41:46,223 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.selectfiles" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_06-T1/selectfiles".
180927-19:41:46,225 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:41:46,258 nipype.workflow DEBUG:
	 [Node] Directory not found "/Vol

180927-19:44:25,249 nipype.workflow DEBUG:
	 Needed dirs: /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_06-T1/split/_report
180927-19:44:25,251 nipype.workflow DEBUG:
	 Removing files: 
180927-19:44:25,377 nipype.workflow DEBUG:
	 saved results in /Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_06-T1/split/result_split.pklz
180927-19:44:25,470 nipype.workflow DEBUG:
	 [Node] Writing post-exec report to "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/_subject_id_06-T1/split/_report/report.rst"
180927-19:44:25,517 nipype.workflow DEBUG:
	 Aggregate: False
180927-19:44:25,625 nipype.workflow INFO:
	 [Node] Finished "b0_flow.split".
180927-19:44:25,628 nipype.workflow INFO:
	 [Node] Setting-up "b0_flow.strip" in "/Volumes/iang/proc/TIGERanalysis/qT1/workflows/b0_flow/strip".
180927-19:44:25,629 nipype.workflow DEBUG:
	 Setting node inputs
180927-19:44:25,645 nipype.workflow DEBUG:
	 [Node] Directory not found "/Volumes/iang/proc/TIGERanalysis

ValueError: BET requires a value for input 'in_file'. For a list of required inputs, see BET.help()

In [ ]:
strip = Node(fsl.BET(out_file = 'skullstripped_b0',
                    output_type = 'NIFTI_GZ'),
            name = 'strip')
#strip uses in_file

In [ ]:
register = Node(fsl.FLIRT(reference = 'skullstripped_b0.nii.gz',
                          out_file = 'FLIRT',
                          output_type = 'NIFTI_GZ',
                          out_matrix_file = 'flirt_transform.mat'),
                name = 'register')